<a href="https://colab.research.google.com/github/Shujaat123/ECM-LSE/blob/master/ECM_LSE_Online.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **ECM-LSE: Prediction of Extracellular Matrix Proteins Using Deep Latent Space Encoding of Composition of k-Spaced Amino Acid Pairs**


This code provide python implementation of ECM-LSE algorithm.

# Loading Useful packages

In [ ]:
## Load useful packages
import sys, os, re, gc
from scipy.io import savemat
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import optimizers
from keras.utils.np_utils import to_categorical
from collections import Counter
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Layer, Input, Dense, BatchNormalization, Dropout
from keras import optimizers
from keras.utils.np_utils import to_categorical
from keras import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score, matthews_corrcoef, balanced_accuracy_score, precision_recall_fscore_support
from keras.models import load_model
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from random import sample
import tensorflow as tf
from keras import regularizers
from keras.constraints import min_max_norm

# Feature-Extraction

The CKSAAP feature encoding calculates the frequency of amino acid pairs separated by any k residues. The CKSAAP encoding scheme reflects the amino acid pair information in small and large range with in the peptides depending upon the value of k(gap). The encoding scheme is utilized from iFeature web server, using the following download link: (https://github.com/Superzchen/iFeature).


In [ ]:
## Define CKSAAP feature-extraction function
def minSequenceLength(fastas):
	minLen = 10000
	for i in fastas:
		if minLen > len(i[1]):
			minLen = len(i[1])
	return minLen

def CKSAAP(fastas, gap=5, **kw):
	if gap < 0:
		print('Error: the gap should be equal or greater than zero' + '\n\n')
		return 0

	if minSequenceLength(fastas) < gap+2:
		print('Error: all the sequence length should be larger than the (gap value) + 2 = ' + str(gap+2) + '\n\n')
		return 0

	AA = 'ACDEFGHIKLMNPQRSTVWY'
	encodings = []
	aaPairs = []
	for aa1 in AA:
		for aa2 in AA:
			aaPairs.append(aa1 + aa2)
	header = ['#']
	for g in range(gap+1):
		for aa in aaPairs:
			header.append(aa + '.gap' + str(g))
	encodings.append(header)
	for i in fastas:
		name, sequence = i[0], i[1]
		code = [name]
		for g in range(gap+1):
			myDict = {}
			for pair in aaPairs:
				myDict[pair] = 0
			sum = 0
			for index1 in range(len(sequence)):
				index2 = index1 + g + 1
				if index1 < len(sequence) and index2 < len(sequence) and sequence[index1] in AA and sequence[index2] in AA:
					myDict[sequence[index1] + sequence[index2]] = myDict[sequence[index1] + sequence[index2]] + 1
					sum = sum + 1
			for pair in aaPairs:
				code.append(myDict[pair] / sum)
		encodings.append(code)
	return encodings

def getSeq(dataset):
	seq=[]
	for index, row in dataset.iterrows():
		array = [row['Class'], row['Sequence']]
		name, sequence = array[0].split()[0], re.sub('[^ARNDCQEGHILKMFPSTWYV-]', '-', ''.join(array[1:]).upper())
		seq.append([name, sequence])
	return seq

def getCKSAAP(seq,Gap):
	cksaapfea = []
	for i in seq:
		temp= CKSAAP([i], gap=Gap)
		cksaapfea.append(temp)

	dt = []
	for i in range(len(cksaapfea)):
		temp = cksaapfea[i][1][1:]
		dt.append(temp)
	dtn = np.array(dt)	
	return dtn

# Build dataset using CKSAAP features
def ExtractFeatures(dataset,Gap):
	seq=[]
	seq = getSeq(dataset)
	features = getCKSAAP(seq,Gap)
	labels = dataset['Class']
	labels[labels=='ECM']=1
	labels[labels=='NON-ECM']=0
	labels = to_categorical(labels)
	return features, labels

# Designing an Auto-Encoder-based classifier model

In [ ]:
## Designing an Auto-Encoder-Classifier model
def ECM_LSE_Final_Model(LV=5,input_size=3600):
    # Encoder Network
    enc_input = Input(shape=(input_size,), name='enc_input')
    enc_l1 = Dense(50, activation='relu', name='encoder_layer1')(enc_input)
    enc_l1 = BatchNormalization()(enc_l1)
    enc_l1 = Dropout(rate=0.3)(enc_l1)

    enc_l2 = Dense(10, activation='relu', name='encoder_layer2')(enc_l1)
    enc_l2 = BatchNormalization()(enc_l2)
    enc_l2 = Dropout(rate=0.3)(enc_l2)

    encoder_output = Dense(LV, activation='sigmoid', name='encoder_output')(enc_l2)

    # Classifier Network
    class_l1 = Dense(10, activation='relu', name='class_layer1')(encoder_output)
    class_l2 = Dense(10, activation='relu', name='class_layer2')(class_l1)
    class_l3 = Dense(10, activation='relu', name='class_layer3')(class_l2)
    class_output = Dense(2, activation='softmax', name='class_output')(class_l3)

    # Decoder Network
    dec_l1 = Dense(10, activation='relu', name='decoder_layer1')(encoder_output)
    dec_l1 = BatchNormalization()(dec_l1)
    dec_l1 = Dropout(0.3)(dec_l1)

    dec_l2 = Dense(50, activation='relu', name='decoder_layer4')(dec_l1)
    dec_l2 = BatchNormalization()(dec_l2)
    dec_l2 = Dropout(0.3)(dec_l2)

    decoder_output = Dense(input_size, activation='sigmoid', name='decoder_output')(dec_l2)

    model = Model(inputs=[enc_input], outputs=[class_output, decoder_output])

    # Compiling model
    model.compile(optimizer='adam',
                  loss={'class_output': 'binary_crossentropy', 'decoder_output': 'mean_squared_error'},
                  loss_weights={'class_output': 0.01, 'decoder_output': 0.99},
                  metrics=[metrics.categorical_accuracy])
    # Here I used adam optimizer with default values, two objective functions are optimized
    # using  weight factors [1 for classifier and 0.1 for decoder loss]
    return model



## Define performance measures

In [ ]:
## Define performance measures
def yoden_index(y, y_pred):
  tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
  j = (tp/(tp+fn)) + (tn/(tn+fp)) - 1
  return j

def pmeasure(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    f1score = 2 * tp / (2 * tp + fp + fn)
    return ({'Sensitivity': sensitivity, 'Specificity': specificity, 'F1-Score': f1score})

def Show_Statistics(msg,Stats):
  print(msg.upper())
  print(70*'-')
  print('Accuracy:',Stats[0])
  print('Sensitivity:',Stats[1])
  print('Specificity:',Stats[2])
  print('F1-Score:',Stats[3])
  print('MCC:',Stats[4])
  print('Balance Accuracy:',Stats[5])
  print('Youden-Index:',Stats[6])
  print('Reconstruction MSE:',Stats[7])
  print(70*'-')

## Loading and pre-processing ECM prediction dataset

In [ ]:
## Loading and pre-processing prediction dataset
data_path = 'https://raw.githubusercontent.com/Shujaat123/ECM-LSE/master/Dataset.csv'


  ## Perform Monte-Carlos Simulations for [num_Trials]\# of independent Trials


In [ ]:
## Perform Monte-Carlos Simulations for [num_Trials]# of independent Trials
Gaps = [0,1,2,3,4,5,6,7,8,9,10]
LVs = [2,3,4,5,6,7,8,9]
num_Trails = 20

for Gap in Gaps:
  dataset_training = pd.read_csv(data_path, index_col=None)
  trainDB_features, trainDB_labels = ExtractFeatures(dataset_training,Gap)

  plist_training = list(np.asarray(np.where(trainDB_labels[:,1]==1)).flatten())
  nlist_training = list(np.asarray(np.where(trainDB_labels[:,1]==0)).flatten())

  for LV in LVs:
    Stats =[]
    for loop_ind in range(0,num_Trails):
        ## train
        p_train = sample(plist_training, 270)
        n_train = sample(nlist_training, 270)
        train_list = p_train + n_train
        X_train = trainDB_features[train_list]
        y_train = trainDB_labels[train_list]

        ## valid
        p_val = list(set(plist_training) - set(p_train))
        p_val = sample(p_val,30)
        n_val = list(set(nlist_training) - set(n_train))
        n_val = sample(n_val,810)
        val_list = p_val + n_val
        X_val = trainDB_features[val_list]
        y_val = trainDB_labels[val_list]   

        ## test
        p_test = list(set(plist_training) - set(p_train) - set(p_val))
        # p_test = sample(p_test,30)
        n_test = list(set(nlist_training) - set(n_train) - set(n_val))
        # n_test = sample(n_test,30)
        test_list = p_test + n_test
        X_test = trainDB_features[test_list]
        y_test = trainDB_labels[test_list]
        
        print('Training Dataset contain:\n', len(p_train),'ECMs\t',len(n_train),'Non-ECMs')
        print('Validation Dataset contain:\n', len(p_val),'ECMs\t',len(n_val),'Non-ECMs')
        print('Test Dataset contain:\n', len(p_test),'ECMs\t',len(n_test),'Non-ECMs')

        model = ECM_LSE_Final_Model(LV,input_size=X_train.shape[1])

        es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=100)

        checkpoint = ModelCheckpoint('models\\model-best' + str(Gap) + '_LV' + str(LV) + '.h5',
                                      verbose=0, monitor='val_class_output_loss',save_best_only=True, mode='auto')


        history = model.fit({'enc_input': X_train},
                            {'class_output': y_train, 'decoder_output': X_train},
                            validation_data = ({'enc_input': X_val},
                            {'class_output': y_val, 'decoder_output': X_val}),
                            epochs=1000, batch_size=100, callbacks=[checkpoint, es], verbose=0)

        del model  # deletes the existing model
        model = load_model('models\\model-best' + str(Gap) + '_LV' + str(LV) + '.h5')

        y_train_pred, X_train_pred = model.predict(X_train,batch_size=540, verbose=0)
        y_train_pred = to_categorical(y_train_pred.argmax(axis=1))
        MSE_X_train_pred = (np.square(X_train_pred - X_train)).mean(axis=1)

        y_val_pred, X_val_pred = model.predict(X_val,batch_size=60, verbose=0)
        y_val_pred = to_categorical(y_val_pred.argmax(axis=1))
        MSE_X_val_pred = (np.square(X_val_pred - X_val)).mean(axis=1)

        y_test_pred, X_test_pred = model.predict(X_test,batch_size=3887, verbose=0)
        y_test_pred = to_categorical(y_test_pred.argmax(axis=1))
        MSE_X_test_pred = (np.square(X_test_pred - X_test)).mean(axis=1)

        
        ## Training Measures
        tr_acc = accuracy_score(y_train.argmax(axis=1), y_train_pred.argmax(axis=1))
        tr_sen = pmeasure(y_train.argmax(axis=1), y_train_pred.argmax(axis=1))['Sensitivity']
        tr_spe = pmeasure(y_train.argmax(axis=1), y_train_pred.argmax(axis=1))['Specificity']
        tr_f1 = pmeasure(y_train.argmax(axis=1), y_train_pred.argmax(axis=1))['F1-Score']
        tr_mcc = matthews_corrcoef(y_train.argmax(axis=1), y_train_pred.argmax(axis=1))
        tr_bacc = balanced_accuracy_score(y_train.argmax(axis=1), y_train_pred.argmax(axis=1))
        tr_yi = yoden_index(y_train.argmax(axis=1), y_train_pred.argmax(axis=1))

        ## Validation Measures
        v_acc = accuracy_score(y_val.argmax(axis=1), y_val_pred.argmax(axis=1))
        v_sen = pmeasure(y_val.argmax(axis=1), y_val_pred.argmax(axis=1))['Sensitivity']
        v_spe = pmeasure(y_val.argmax(axis=1), y_val_pred.argmax(axis=1))['Specificity']
        v_f1 = pmeasure(y_val.argmax(axis=1), y_val_pred.argmax(axis=1))['F1-Score']
        v_mcc = matthews_corrcoef(y_val.argmax(axis=1), y_val_pred.argmax(axis=1))
        v_bacc = balanced_accuracy_score(y_val.argmax(axis=1), y_val_pred.argmax(axis=1))
        v_yi = yoden_index(y_val.argmax(axis=1), y_val_pred.argmax(axis=1))

        ## Test Measures
        t_acc = accuracy_score(y_test.argmax(axis=1), y_test_pred.argmax(axis=1))
        t_sen = pmeasure(y_test.argmax(axis=1), y_test_pred.argmax(axis=1))['Sensitivity']
        t_spe = pmeasure(y_test.argmax(axis=1), y_test_pred.argmax(axis=1))['Specificity']
        t_f1 = pmeasure(y_test.argmax(axis=1), y_test_pred.argmax(axis=1))['F1-Score']
        t_mcc = matthews_corrcoef(y_test.argmax(axis=1), y_test_pred.argmax(axis=1))
        t_bacc = balanced_accuracy_score(y_test.argmax(axis=1), y_test_pred.argmax(axis=1))
        t_yi = yoden_index(y_test.argmax(axis=1), y_test_pred.argmax(axis=1))


        Stats.append([tr_acc, tr_sen, tr_spe, tr_f1, tr_mcc, tr_bacc, tr_yi, -10*np.log10(MSE_X_train_pred.mean()),
                      v_acc, v_sen, v_spe, v_f1, v_mcc, v_bacc, v_yi, -10*np.log10(MSE_X_val_pred.mean()),
                      t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi, -10*np.log10(MSE_X_test_pred.mean())])
        print('Gap(k)=',Gap,'LV=',LV,'Trial:',loop_ind,'Training-Youden index:',tr_yi, 'Validation-Youden index:',v_yi, 'Test-Youden index:',t_yi)
    Statistics = np.asarray(Stats)
    filename = 'ECM_LSE_results_GAP' + str(Gap) + '_LV' + str(LV) + 'x.mat'
    savemat(filename,{'Statistics':Statistics})


## Show Classification/Reconstruction Statistics

In [ ]:
## Show Classification/Reconstruction Statistics
Show_Statistics('Training Results (MEAN)',Statistics.mean(axis=0)[0:8])
Show_Statistics('Validation Results (MEAN)',Statistics.mean(axis=0)[8:16])
Show_Statistics('Test Results (MEAN)',Statistics.mean(axis=0)[16:24])